In [1]:
from latools import *

import pandas as pd

    # if 'meta_regex' in dataformat.keys():
    #     meta = Bunch()
    #     for k, v in dataformat['meta_regex'].items():
    #         out = re.search(v[-1], lines[int(k)]).groups()
    #         for i in np.arange(len(v[0])):
    #             meta[v[0][i]] = out[i]

    # # sample name
    # if name_mode == 'file_names':
    #     sample = os.path.basename(data_file).split('.')[0]
    # elif name_mode == 'metadata_names':
    #     sample = meta['name']
    # else:
    #     sample = name_mode

    # # column and analyte names
    # columns = np.array(lines[dataformat['column_id']['name_row']].strip().split(
    #     dataformat['column_id']['delimiter']))
    # if 'pattern' in dataformat['column_id'].keys():
    #     pr = re.compile(dataformat['column_id']['pattern'])
    #     analytes = [pr.match(c).groups()[0] for c in columns if pr.match(c)]

    # # do any required pre-formatting
    # if 'preformat_replace' in dataformat.keys():
    #     with open(data_file) as f:
    #         fbuffer = f.read()
    #     for k, v in dataformat['preformat_replace'].items():
    #         fbuffer = re.sub(k, v, fbuffer)
    #     # dead data
    #     read_data = np.genfromtxt(BytesIO(fbuffer.encode()),
    #                               **dataformat['genfromtext_args']).T
    # else:
    #     # read data
    #     read_data = np.genfromtxt(data_file,
    #                               **dataformat['genfromtext_args']).T

    # # data dict
    # dind = np.ones(read_data.shape[0], dtype=bool)
    # dind[dataformat['column_id']['timecolumn']] = False

    # data = Bunch()
    # data['Time'] = read_data[dataformat['column_id']['timecolumn']]

    # # convert raw data into counts
    # # TODO: Is this correct? Should actually be per-analyte dwell?
    # if 'unit' in dataformat:
    #     if dataformat['unit'] == 'cps':
    #         tstep = data['Time'][1] - data['Time'][0]
    #         read_data[dind] *= tstep
    #     else:
    #         pass
    # data['rawdata'] = Bunch(zip(analytes, read_data[dind]))
    # data['total_counts'] = read_data[dind].sum(0)

    # return sample, analytes, data, meta
from sklearn import cluster as cl

%matplotlib inline

# TODO:
https://github.com/oscarbranson/latools/issues

1. Drift Correction - Warn if variation between SRMs for a particular element is greater than STD?
2. Despiking: repeat until no changes are made?
3. Make SRM ID smarter, so it doesn't need to know where the SRMs are?

*Less Important*
1. Incorporate laser log file

In [2]:
dat = analyse('latools/resources/test_data')

dat.despike()
dat.autorange()
dat.bkg_calc_weightedmean(n_min=10)
dat.bkg_subtract()
dat.ratio(internal_standard='Ca43')
dat.calibrate()

AutoRange:  20%|██        | 1/5 [00:00<00:00,  8.06it/s]

Starting analysis using "UCD-AGILENT" configuration:
  5 Data Files Loaded: 2 standards, 3 samples
  Analytes: Mg24 Mg25 Al27 Ca43 Ca44 Mn55 Sr88 Ba137 Ba138
  Internal Standard: Ca43


Applying Calibrations: 100%|██████████| 5/5 [00:00<00:00, 11.82it/s]


In [4]:
dat.minimal_export()

In [3]:
dat.log

["__init__ :: args=() kwargs={'srm_file': None, 'time_format': None, 'srm_identifier': 'STD', 'dataformat': None, 'config': 'DEFAULT', 'data_folder': 'latools/resources/test_data', 'names': 'file_names', 'internal_standard': 'Ca43', 'extension': '.csv', 'errorhunt': False, 'cmap': None}",
 'despike :: args=() kwargs={}',
 'autorange :: args=() kwargs={}',
 "bkg_calc_weightedmean :: args=() kwargs={'n_min': 10}",
 'bkg_subtract :: args=() kwargs={}',
 "ratio :: args=() kwargs={'internal_standard': 'Ca43'}",
 'calibrate :: args=() kwargs={}']

## Improve SRM Identification

In [5]:
self = dat

In [141]:
# def srm_id_auto(self, srms_used=['NIST610', 'NIST612', 'NIST614']):

srms_used=['NIST610', 'NIST612', 'NIST614']

# compile mean and standard errors of samples
for s in self.data:
    stdtab = pd.DataFrame(columns=pd.MultiIndex.from_product([s.analytes, ['err','mean']]))
    stdtab.index.name = 'uTime'

    for n in range(1,s.n+1):
        ind = s.ns == n
        for a in s.analytes:
            aind = ind & ~np.isnan(nominal_values(s.focus[a]))
            stdtab.loc[np.nanmean(s.uTime[s.ns == n]),
                   (a, 'mean')] = np.nanmean(s.focus[a][aind])
            stdtab.loc[np.nanmean(s.uTime[s.ns == n]),
                       (a, 'err')] = np.nanmean(s.focus[a][aind]) / np.sqrt(sum(aind))

    # sort column multiindex
    stdtab = stdtab.loc[:,stdtab.columns.sort_values()]
    # sort row index
    stdtab.sort_index(inplace=True)

    # create 'SRM' column for naming SRM
    stdtab.loc[:,'SRM'] = ''
    stdtab.loc[:,'STD'] = s.sample

    s.stdtab = stdtab

stdtab = pd.concat([s.stdtab for s in self.data]).apply(pd.to_numeric, 1, errors='ignore')

# load SRM info
srmdat = pd.read_csv(self.srmfile)
srmdat.set_index('SRM', inplace=True)
srmdat = srmdat.loc[srms_used]

# isolate measured elements
elements = np.unique([re.findall('[A-Za-z]+', a)[0] for a in self.analytes])
srmdat = srmdat.loc[srmdat.Item.apply(lambda x: any([a in x for a in elements]))]
# label elements
srmdat.loc[:,'element'] = np.nan
for e in elements:
    srmdat.loc[srmdat.Item.str.contains(e),'element'] = e

# convert to table in same format as stdtab
srm_tab = srmdat.loc[:,['Value','element']].reset_index().pivot(index='SRM', columns='element', values='Value')

# Auto-ID STDs
# 1. identify elements in measured SRMS with biggest range of values
meas_tab = stdtab.loc[:,(slice(None),'mean')]  # isolate means of standards
meas_tab.columns = meas_tab.columns.droplevel(1)  # drop 'mean' column names
meas_tab.columns = [re.findall('[A-Za-z]+', a)[0] for a in meas_tab.columns]  # rename to element names
meas_tab = meas_tab.T.groupby(level=0).first().T  # remove duplicate columns

ranges = nominal_values(meas_tab.apply(lambda a: np.ptp(a) / np.nanmean(a), 0))  # calculate relative ranges of all elements
# (used as weights later)

# 2. Work out which standard is which
# normalise all elements between 0-1
def normalise(a):
    a = nominal_values(a)
    a[a==1] = np.nan
    if np.nanmin(a) < np.nanmax(a):
        return (a - np.nanmin(a)) / np.nanmax(a - np.nanmin(a))
#         return a / np.nansum(a)
    else:
        return np.ones(a.shape)

nmeas = meas_tab.apply(normalise, 1)
nmeas.replace(np.nan,1, inplace=True)
nsrm_tab = srm_tab.apply(normalise, 1)
nsrm_tab.replace(np.nan,1, inplace=True)

matches = pd.DataFrame(columns=srms_used, index=nmeas.index)

for uT, r in nmeas.iterrows():  # for each standard...
    idx = abs((nsrm_tab - r) * ranges).sum(1)
    matches.loc[uT] = idx
#     # calculate the absolute difference between the normalised elemental
#     # values for each measured SRM and the SRM table. Each element is
#     # multiplied by the relative range seen in that element (i.e. range / mean
#     # measured value), so that elements with a large difference are given
#     # more importance in identifying the SRM.
#     # This produces a table, where each row contains the difference between
#     # a known vs. measured SRM. The measured SRM is identified as the SRM that
#     # has the smallest weighted sum value.
#     stdtab.loc[uT, 'SRM'] = srm_tab.index[idx == min(idx)].values[0]
# stdtab = stdtab.reset_index().set_index(['STD','SRM','uTime'])


# # combine to make SRM reference tables
# srmtabs = {}
# for a in self.analytes:
#     el = re.findall('[A-Za-z]+',a)[0]

#     sub = stdtab.loc[:,a]

#     srmsub = srmdat.loc[srmdat.element==el, ['Value','Uncertainty']]

#     srmtab = sub.join(srmsub)
#     srmtab.columns = ['meas_err', 'meas_mean', 'srm_mean', 'srm_err']

#     srmtabs[a] = srmtab

# self.srmtabs = pd.concat(srmtabs).apply(nominal_values)
#     return

In [142]:
matches

,NIST610,NIST612,NIST614
uTime,,,
358.992770,6.62538,6.57121,6.58359
430.755820,6.58627,6.58259,6.60996
475.741684,5.93376,5.88435,5.91173
524.297882,7.00552,6.96478,6.99216
612.070306,7.28992,7.31203,7.3394
660.269397,7.12949,7.12863,7.156
704.719750,6.24845,6.32324,6.35062
744.885610,6.96652,7.14097,7.16835
783.266386,6.62519,6.72315,6.75052


In [125]:
i = 744
a = nmeas.loc[abs(nmeas.index - i) == min(abs(nmeas.index - i))]
uT, r = list(a.iterrows())[0]

In [126]:
abs((nsrm_tab - r) * ranges).sum(1)

SRM
NIST610    6.966515
NIST612    7.140972
NIST614    7.168348
dtype: float64

In [87]:
idx

NIST610             NaN
NIST612             NaN
NIST614             NaN
1282.576173611111   NaN
dtype: float64

In [40]:
nsrm_tab

element,Al,Ba,Ca,Mg,Mn,Sr
SRM,,,,,,
NIST610,0.092608,0.0,1.0,0.007763,0.002360,0.001277
NIST612,0.093699,0.0,1.0,0.001184,0.000197,0.000287
NIST614,0.094274,0.0,1.0,0.000644,0.000001,0.000235


In [41]:
nmeas

,Al,Ba,Ca,Mg,Mn,Sr
uTime,,,,,,
358.992770,0.011644,0.0,0.635373,1.000000,0.001187,0.731220
430.755820,0.009464,0.0,0.578547,1.000000,0.005189,0.726794
475.741684,0.014727,0.0,0.585887,1.000000,0.002394,0.500850
524.297882,0.009564,0.0,0.525098,0.733227,0.002924,1.000000
612.070306,0.019911,0.0,0.636565,0.966650,0.006765,1.000000
660.269397,0.016953,0.0,0.582353,0.844997,0.005362,1.000000
704.719750,0.015370,0.0,0.743097,1.000000,0.009985,0.630590
744.885610,0.019681,0.0,0.940868,1.000000,0.016076,0.898089
783.266386,0.056886,0.0,0.549327,0.619683,0.011401,1.000000


In [4]:
dat.srm_id_auto()

In [5]:
self = dat

In [6]:
from scipy import odr

## Detection Limit Calculation

## Data Format Tester

In [35]:
import latools as la
import configparser
import json
import os, re
import numpy as np
import pkg_resources as pkgrs
from latools.helpers.helpers import Bunch

from shutil import copyfile

In [94]:
config = la.config.read_configuration()
fdataformat = './UCD_dataformat.json'

In [98]:
with open(fdataformat) as f:
    dataformat = json.load(f)

In [99]:
dataformat

{'column_id': {'delimiter': ',',
  'name_row': 3,
  'pattern': '([A-z]{1,2}[0-9]{1,3})',
  'timecolumn': 0},
 'genfromtext_args': {'delimiter': '2,', 'skip_header': 4},
 'meta_regex': {'0': [['path'], '(.*)'],
  '2': [['date', 'method'],
   '([A-Z][a-z]+ [0-9]+ [0-9]{4}[ ]+[0-9:]+ [amp]+).* ([A-z0-9]+\\.m)']},
 'unit': 'cps'}

In [11]:
ls 'latools/resources/test_data/'

Sample-1.csv  Sample-2.csv  Sample-3.csv  STD-1.csv  STD-2.csv


In [105]:
%load_ext autoreload
%autoreload 2

from latools.helpers.config import test_dataformat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
test_dataformat('latools/resources/test_data/STD-1.csv', fdataformat, 'file_names')

*************************************************
Testing suitability of data format description...
  Dataformat File: ./UCD_dataformat.json
  Data File: latools/resources/test_data/STD-1.csv
*************************************************
  Test: open data file...
    Success!
  Test: read dataformat file...
    Success!
  Test: read metadata using 'metadata_regex'...
    Line 2: date: Oct 29 2015  03:06:01 pm
    Finished - does the above look correct?
  Test: Sample Name IDs...
    Sample name grabbed from file: STD-1
        ***Is the sample name correct?***
  Test: Getting Column Names...
    Columns from line 3: Time [Sec], Mg24, Mg25, Al27, Ca43, Ca44, Mn55, Sr88, Ba137, Ba138
    Cleaning up using column_id/pattern...
    Cleaned Analyte Names: Ca43, Ca44, Ba137, Ba138
        ***This should only contain analyte names... does it?***
  Test: Reading Data...
    checking dimensions...
        ***PROBLEM: There are 10 data columns, but 5 column names.
        Check your identifi

ValueError: Data - Column Name mismatch